# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [40]:
import ames_fe # The module for loading external data - Ames Housing dataset
import os
import pandas as pd
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Workspace, Dataset, Experiment, Model
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.train.automl import AutoMLConfig

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

quick-starts-ws-152755
aml-quickstarts-152755
southcentralus
a0a76bad-11a1-4a2d-9887-97a29122c8ed


In [3]:
# Create compute cluster
# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [17]:
train, test = ames_fe.load_data_tailored()
print(f"train.shape = {train.shape}, test.shape = {test.shape}")

train.shape = (1460, 78), test.shape = (1459, 78)


In [18]:
# Register dataset
blob = ws.get_default_datastore()
train_reg = TabularDatasetFactory.register_pandas_dataframe(train, blob, name='Ames-housing-dataset',
                                                          description='Ames Housing training data.')

Method register_pandas_dataframe: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/070139d9-b7de-4d14-86fc-30a1fcb87b8b/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [19]:
# Choose a name for experiment
experiment_name = 'Ames-housing-AutoML'

experiment=Experiment(ws, experiment_name)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [23]:
# Set parameters for AutoMLConfig
automl_settings = {
    'experiment_timeout_minutes' : 120,
    'n_cross_validations' : 3,
    'enable_early_stopping' : True,
    'iteration_timeout_minutes' : 5,
    'max_concurrent_iterations' : 4,
    'max_cores_per_iteration' : -1
}

automl_config = AutoMLConfig(
    task='regression',
    primary_metric='normalized_root_mean_squared_error',
    compute_target=cpu_cluster,
    training_data=train_reg,
    label_column_name='SalePrice',
    **automl_settings)

In [24]:
from azureml.widgets import RunDetails
# TODO: Submit your experiment
aml_run = experiment.submit(automl_config)
RunDetails(aml_run).show()
aml_run.wait_for_completion(show_output=True)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
Ames-housing-AutoML,AutoML_1eb2883c-ec84-44fa-b028-c77e1f99ce4b,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
Ames-housing-AutoML,AutoML_1eb2883c-ec84-44fa-b028-c77e1f99ce4b,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feature h

{'runId': 'AutoML_1eb2883c-ec84-44fa-b028-c77e1f99ce4b',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-07-28T04:41:33.366858Z',
 'endTimeUtc': '2021-07-28T05:04:00.401664Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'normalized_root_mean_squared_error',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"Ames-housing-AutoML","subscription_id":"a0a76bad-11a1-4a2d-9887-97a29122c8ed","resource_group":"aml-quickstarts-152755","workspace_name":"quick-starts-ws-152755","region":"southcentralus","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":100

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [25]:
# Get your best run and save the model from that run.
best_run, fitted_model = aml_run.get_output()
print(best_run)
print(fitted_model)

Package:azureml-automl-runtime, training version:1.32.0, current version:1.31.0
Package:azureml-core, training version:1.32.0, current version:1.31.0
Package:azureml-dataset-runtime, training version:1.32.0, current version:1.31.0
Package:azureml-defaults, training version:1.32.0, current version:1.31.0
Package:azureml-interpret, training version:1.32.0, current version:1.31.0
Package:azureml-mlflow, training version:1.32.0, current version:1.31.0
Package:azureml-pipeline-core, training version:1.32.0, current version:1.31.0
Package:azureml-telemetry, training version:1.32.0, current version:1.31.0
Package:azureml-train-automl-client, training version:1.32.0, current version:1.31.0
Package:azureml-train-automl-runtime, training version:1.32.0, current version:1.31.0


Run(Experiment: Ames-housing-AutoML,
Id: AutoML_1eb2883c-ec84-44fa-b028-c77e1f99ce4b_36,
Type: azureml.scriptrun,
Status: Completed)
RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='regression', working_dir='/mnt/batch/ta...
)), ('elasticnet', ElasticNet(alpha=0.05357894736842105, copy_X=True, fit_intercept=True, l1_ratio=0.6352631578947369, max_iter=1000, normalize=False, positive=False, precompute=False, random_state=None, selection='cyclic', tol=0.0001, warm_start=False))], verbose=False))], weights=[0.6153846153846154, 0.07692307692307693, 0.07692307692307693, 0.07692307692307693, 0.15384615384615385]))],
                           

In [26]:
best_run_metrics = best_run.get_metrics()
best_run_metrics

{'root_mean_squared_log_error': 0.12855427357163166,
 'spearman_correlation': 0.9577828532398551,
 'normalized_root_mean_squared_log_error': 0.04181709823007532,
 'r2_score': 0.894495574186684,
 'root_mean_squared_error': 25739.296393934364,
 'mean_absolute_percentage_error': 9.05916141769014,
 'normalized_mean_absolute_error': 0.02133548496494304,
 'normalized_root_mean_squared_error': 0.03574405831680928,
 'explained_variance': 0.8945550043138528,
 'mean_absolute_error': 15363.68272325548,
 'median_absolute_error': 9955.667583551089,
 'normalized_median_absolute_error': 0.013825395894391178,
 'residuals': 'aml://artifactId/ExperimentRun/dcid.AutoML_1eb2883c-ec84-44fa-b028-c77e1f99ce4b_36/residuals',
 'predicted_true': 'aml://artifactId/ExperimentRun/dcid.AutoML_1eb2883c-ec84-44fa-b028-c77e1f99ce4b_36/predicted_true'}

In [27]:
import json
with open('best_run_metrics.json', 'w') as file:
    json.dump(best_run_metrics, file)
with open('best_run_details.txt', 'w') as file:
    file.write(str(best_run.get_details()))

In [28]:
# Test the best model accuracy in the Kaggle competition
test.shape

(1459, 78)

In [29]:
test.drop(columns=['SalePrice'], inplace=True)

In [30]:
# NaN values not treated by the model .... 
y_hat = fitted_model.predict(test.fillna(0))
y_hat[:10]

array([123002.58819187, 156797.34398885, 188204.69635731, 193086.97599697,
       194323.37378999, 176278.44967357, 173645.13246914, 162750.2912638 ,
       187465.80612224, 126787.02080805])

In [31]:
len(y_hat)

1459

In [32]:
with open('automl-submission.csv', 'w') as file:
    pd.DataFrame({'Id' : test.index.to_list(), 'SalePrice': y_hat}).to_csv(file, index=False)

In [33]:
os.listdir()

['.amlignore',
 '.amlignore.amltmp',
 '.azureml',
 '.config',
 '.ipynb_aml_checkpoints',
 '.ipynb_checkpoints',
 'ames.py',
 'ames_fe.py',
 'ames_fe.py.amltmp',
 'automl-submission.csv',
 'automl.ipynb',
 'automl.log',
 'azureml_automl.log',
 'best_run_details.txt',
 'best_run_metrics.json',
 'data_description.txt',
 'house-prices-advanced-regression-techniques.zip',
 'Register_dataset.ipynb',
 'sample_submission.csv',
 'test.csv',
 'train.csv',
 '__pycache__']

In [34]:
# Set the enviroment variables
os.environ['KAGGLE_USERNAME'] = "lubomrstraka"
os.environ['KAGGLE_KEY'] = "c7347462ef834e6645ce238c2f2fa561"
! kaggle competitions submit -c house-prices-advanced-regression-techniques -f automl-submission.csv -m "Submission generated by Azure Automated ML."

100%|██████████████████████████████████████| 33.6k/33.6k [00:01<00:00, 28.5kB/s]
Successfully submitted to House Prices - Advanced Regression Techniques

>2021-07-23: Your submission scored 0.13612, which is not an improvement of your best score (0.12481).

>2021-07-28: Your submission scored 0.13699, which is not an improvement of your best score (0.12481).

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [37]:
# Check the path to the model
for i,n in enumerate(best_run.get_file_names()):
    print(i,n)

0 automl_driver.py
1 azureml-logs/55_azureml-execution-tvmps_b0970c78538d214ccf585b66041762ee0fdd2d587fc581545922574cf255b1a8_d.txt
2 azureml-logs/65_job_prep-tvmps_b0970c78538d214ccf585b66041762ee0fdd2d587fc581545922574cf255b1a8_d.txt
3 azureml-logs/70_driver_log.txt
4 azureml-logs/75_job_post-tvmps_b0970c78538d214ccf585b66041762ee0fdd2d587fc581545922574cf255b1a8_d.txt
5 azureml-logs/process_info.json
6 azureml-logs/process_status.json
7 explanation/2cedcf07/eval_data_viz.interpret.json
8 explanation/2cedcf07/expected_values.interpret.json
9 explanation/2cedcf07/features.interpret.json
10 explanation/2cedcf07/global_names/0.interpret.json
11 explanation/2cedcf07/global_rank/0.interpret.json
12 explanation/2cedcf07/global_values/0.interpret.json
13 explanation/2cedcf07/local_importance_values.interpret.json
14 explanation/2cedcf07/rich_metadata.interpret.json
15 explanation/2cedcf07/true_ys_viz.interpret.json
16 explanation/2cedcf07/visualization_dict.interpret.json
17 explanation/a3f4

In [45]:
# Save the best model
os.makedirs('./outputs/', exist_ok=True)
for i in range(32,39):
    print(best_run.get_file_names()[i])
    best_run.download_file(best_run.get_file_names()[i], output_file_path='./outputs/')

outputs/conda_env_v_1_0_0.yml
outputs/env_dependencies.json
outputs/internal_cross_validated_models.pkl
outputs/model.pkl
outputs/pipeline_graph.json
outputs/scoring_file_v_1_0_0.py
outputs/scoring_file_v_2_0_0.py


In [42]:
model = Model.register(ws, model_path='outputs/model.pkl', model_name='Ames-Housing-AutoML-Model', tags=best_run_metrics)
print(model.name, model.id, model.version, sep='\t')

Registering model Ames-Housing-AutoML-Model
Ames-Housing-AutoML-Model	Ames-Housing-AutoML-Model:1	1


In [43]:
from azureml.core.webservice import AciWebservice
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1,
                                              memory_gb=1,
                                              tags={"data" : "Kaggle", "method" : "AutoML"},
                                              description="Predict Ames Housing Prices")

In [46]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

env = Environment("project-env")
cd = CondaDependencies('outputs/conda_env_v_1_0_0.yml')
env.python.conda_dependencies = cd
# Register environment to re-use later
env.register(workspace=ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210531.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "project-env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
  

In [ ]:
%%time
import uuid
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model

ws = Workspace.from_config()
model = Model(ws, 'Ames-Housing-AutoML-Model')

myenv = Environment.get(workspace=ws, name="project-env", version="1")
inference_config = InferenceConfig(entry_script="outputs/scoring_file_v_1_0_0.py", environment=myenv)

service_name = 'ames-housing-aml-' + str(uuid.uuid4())[:4]
service = Model.deploy(workspace=ws,
                      name=service_name,
                      models=[model],
                      inference_config=inference_config,
                      deployment_config=aciconfig)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-07-28 05:43:39+00:00 Creating Container Registry if not exists..
2021-07-28 05:43:49+00:00 Registering the environment..
2021-07-28 05:43:51+00:00 Building image.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

In [46]:
# Delete() is used to deprovision and delete the AmlCompute target. 
cpu_cluster.delete()